In [1]:
import tensorflow as tf
import numpy as np
import shelve
import joblib
from math import ceil
import time
import layers
import model
from Augment import valid_datagen
import helper
from test_config import vocabulary,infer_batch_size,mount_point,input_dir,resume_epoch,img_height,img_width
from Arch import CNN

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
graph,dropout_lstm,dropout_fc,inputs,time_steps,targets,loss,train,decoded,label_error_rate,seq_len,is_training,conv_dropout,gradients,interim_dropout= model.model()

valid_generator = valid_datagen.flow_from_directory(mount_point+input_dir,target_size=(img_height,img_width),color_mode='grayscale',batch_size = infer_batch_size)

num_vbatches = 1

++Conv layer info info:
Num_filters =  64
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 2)
++Conv layer info info:
Num_filters =  128
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_1:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 2)
++Conv layer info info:
Num_filters =  256
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_2:0", shape=(), dtype=float32)
++Conv layer info info:
Num_filters =  256
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_3:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 1)
++Conv layer info info:
Num_filters =  512
Ker

INFO:tensorflow:Summary name stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/kernel:0-weight is illegal; using stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/kernel_0-weight instead.
INFO:tensorflow:Summary name stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/bias:0-weight is illegal; using stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/bias_0-weight instead.
INFO:tensorflow:Summary name dense/kernel:0-weight is illegal; using dense/kernel_0-weight instead.
INFO:tensorflow:Summary name dense/bias:0-weight is illegal; using dense/bias_0-weight instead.
Found 1 images belonging to 2 classes.


In [3]:
infer_inputs = []; infer_outputs = []; infer_targets = []

In [4]:
with tf.Session(graph = graph) as sess:
        
    saver = tf.train.Saver(max_to_keep=None)
    
    sess.run(tf.global_variables_initializer())
    
    timer  = 0

    saver.restore(sess, mount_point+'saved_models_ler0.07_leakyrelu/cnn_lstm_fc_'+str(resume_epoch))

    start_time = time.time()

    count = 0

    for x,y in valid_generator:
       
        infer_inputs.append(x)
        
        print(y)
        
        print(x.shape)
        
        actual_batch_size = x.shape[0]
        
        widths = [189]
        
        if count == num_vbatches:
            break

        feed = {
                     inputs:x,
                     time_steps:widths,conv_dropout:[1]*len(CNN),
                     dropout_fc:1,dropout_lstm:1,interim_dropout:1,
                     is_training:False
                }

        d = sess.run([decoded],feed_dict=feed)
        infer_outputs.append(d)
        count+=1

INFO:tensorflow:Restoring parameters from ../saved_models_ler0.07_leakyrelu/cnn_lstm_fc_37
[[0. 1.]]
(1, 114, 758, 1)
[[0. 1.]]
(1, 114, 758, 1)


In [5]:
infer_outputs[0][0][0][1]

array([ 0, 35,  0, 57, 68, 68,  0, 55, 67, 67, 70, 72, 53, 61, 66, 71,  0,
       28, 16,  0, 31, 47, 44, 41, 30,  0, 29, 41, 70, 44, 30,  0])

In [6]:
content = "".join([vocabulary[char] for char in infer_outputs[0][0][0][1]])

In [7]:
print(content)

 I epp coortains B2 EUROD COrRD 


In [8]:
img1 = infer_inputs[0][0]
img1 = img1.reshape(img1.shape[:2])

In [9]:
img1.shape

(114, 758)

In [10]:
plt.imsave('test_input.jpg',img1,cmap='gray',format='jpg')

NameError: name 'plt' is not defined